 # Project 7 - Implémentation d'un Modèle de Scoring
 
 ## Part 1 of XX - Prétraitment y inclu Aggrégation et Feature Engineering

# 1. Kaggle Resources

**Data:**
* https://www.kaggle.com/c/home-credit-default-risk/data

**Series of Notebooks detailing the Problem:**
* https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction
* https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering
* https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering-p2
* ...

**EDA:**
* https://www.kaggle.com/gpreda/home-credit-default-risk-extensive-eda

**Feature Engineering:**
* https://www.kaggle.com/jsaguiar/lightgbm-with-simple-features

**Models/Scripts - Light_GBM**
* https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
* https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm

**DNN:**
https://www.kaggle.com/shep312/deep-learning-in-tf-with-upsampling-lb-758


# 2. Imports Bibliothèques

In [1]:
import os, re, gc

import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
import sys
import win32com.client 

# 3. Fonctions

## 3.1 Traitement des Features Catégorielles

In [4]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    orig_columns = df.columns
    cat_columns = df.select_dtypes(include = 'object').columns
    df = pd.get_dummies(df, columns=cat_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in orig_columns]
    #Apply re.sub and replace method to remove punctuation symbols and transform spaces into underscores.
    df.rename(columns={col : re.sub(r'[^\w\s]', '', col).replace(' ','_') for col in new_columns}, inplace=True)
    new_columns = [c for c in df.columns if c not in orig_columns]
    return df, new_columns

In [5]:
def cat_encoder(series, cat_type=None, impute=True, impute_value=None,):
    
    if cat_type == 'education':
        repl_dict = {'lower secondary' : 0,
                     'secondary / secondary special' : 1, 
                     'incomplete higher' : 2, 
                     'higher education' : 3,
                     'academic degree' : 4}
    elif cat_type == 'weekday':
        repl_dict = {'monday' : 0,
                     'tuesday' : 1, 
                     'wednesday' : 2, 
                     'thursday' : 3,
                     'friday' : 4,
                     'saturday' : 5,
                     'sunday' : 6}
    elif cat_type == 'yes_no' :
        list_one  = ['y', 'yes', 'true', 't']
        list_zero = ['n', 'no', 'false', 'f']
        repl_dict = {i:1 for i in list_one} | {i:0 for i in list_zero}
    elif cat_type == 'gender' :
        list_one  = ['f', 'fem', 'female', 'w', 'woman']
        list_zero = ['m', 'man', 'male']
        repl_dict = {i:1 for i in list_one} | {i:0 for i in list_zero}
    elif cat_type == 'loan' :
        repl_dict = {'cash loans':1,
                     'revolving loans':0,}
    elif cat_type == 'currency':
        list_one  = ['currency 1']
        list_zero = ['currency 2', 'currency 3', 'currency 4']
        repl_dict = {i:1 for i in list_one} | {i:0 for i in list_zero}
    elif cat_type == 'yield_group' :
        repl_dict = {'low_action' : 1,
                     'low_normal' : 2,
                     'middle' : 3,
                     'high' : 4}
    else:
        le = LabelEncoder()
        s = le.fit_transform(series)
        s = pd.Series(s, name=series.name)
        repl_dict = {le.inverse_transform([v])[0]:v for v in s.unique()}
        return s
    
    s = series.str.lower().replace(repl_dict)
    if impute and (len(s.unique()) > len(set(repl_dict.values()))) :
        if impute_value == None:
            impute_value = np.nan
        s = pd.Series([impute_value if value not in repl_dict.values() else value for value in s], name=series.name)
    return s, repl_dict

In [6]:
def specific_cat_reduction(series, cat_type=None):
    if cat_type == 'bureau_credit_type':
        categories =['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
                     'Microloan', 'Loan for business development', 'Unknown type of loan',
                     'Another type of loan']
        other_value = 'Another type of loan'
        s = series.fillna('Unknown type of loan')
        s = pd.Series([v if v in categories else other_value for v in s], name=series.name)
        return s
    elif cat_type == 'bureau_balance_status':
        cat_dict = {'C' : -1, 'X' : np.nan}
        s = pd.to_numeric(series.replace(cat_dict))
        return s
    elif cat_type =='loan_purpose' :
        cat_list_dict = {'Mobile' : ['Mobile'],
                         'Electronics' : ['Consumer Electronics', 'Purchase of electronic equipment'],
                         'Computers' : ['Computers'],
                         'Audio/Video' : ['Audio/Video'],
                         'Furniture' : ['Furniture'],
                         'Photo / Cinema Equipment' : ['Photo / Cinema Equipment'],
                         'Construction Materials' : ['Construction Materials'],
                         'Repairs' : ['Repairs'],
                         'Clothing and Accessories' : ['Clothing and Accessories'],
                         'Other' : ['Other', 'Animals', 'Money for a third person',
                                    'Direct Sales', 'Business development', 'Additional Service', 'Insurance',
                                    'Weapon'],
                         'Urgent needs' : ['Urgent needs'],
                         'Car repairs / accessories' : ['Auto Accessories', 'Car repairs'],
                         'Jewelry' : ['Jewelry'],
                         'Homewares' : ['Homewares'],
                         'Medical Supplies' : ['Medical Supplies', 'Medicine'],
                         'Vehicles' : ['Vehicles', 'Buying a used car', 'Buying a new car'],
                         'Sport and Leisure' : ['Sport and Leisure', 'Fitness', 'Hobby'],
                         'House Construction / Purchase' : ['House Construction', 'Building a house or an annex', 
                                                            'Buying a home', 'Buying a holiday home / land',
                                                            'Gasification / water supply', 'Buying a garage'],
                         'Gardening' : ['Gardening'],
                         'Everyday expenses' : ['Everyday expenses'],
                         'Office Appliances' : ['Office Appliances'],
                         'Payments on other loans' : ['Payments on other loans'],
                         'Travel / Tourism / Wedding' : ['Tourism', 'Journey', 'Wedding / gift / holiday'],
                         'Education' : ['Education'],
                         #'Wedding / gift / holiday' : ['Wedding / gift / holiday'],
                         #'Direct Sales' : ['Direct Sales'],
                         #'Business development' : ['Business development'],
                         #'Services' : ['Additional Service', 'Insurance'],
                         #'Weapon' : ['Weapon'],
                         }
        cats = cat_list_dict.keys()
        repl_dict = {}
        for k, l in cat_list_dict.items():
            if len(l) > 1 :
                repl_dict = repl_dict | {el:k for el in l}
        s = series.replace(repl_dict)
        repl_dict = {el:np.nan for el in s.unique() if el not in cats}
        s = s.replace(repl_dict)
        return s
    elif cat_type == 'suite_type':
        repl_dict = {'Group of people' : 'Other',
                     'Other_B' : 'Other',
                     'Other_A' : 'Other',
                    }
        s = series.replace(repl_dict).fillna('Other')
        return s
    elif cat_type == 'payment_type':
        repl_dict = {'Cash through the bank' : 'Cash',
                     'Non-cash from your account': 'Cashless',
                     'Cashless from the account of the employer' : 'Cashless'}
        s = series.replace(repl_dict)
        return s
    else:
        return series

## 3.2 Premier Prétraitement et Aggregation

### 3.2.1 Le fichier: application

In [7]:
def preprocess_application(filepath, num_rows=None, nan_as_category=True) :
    df = pd.read_csv(filepath, nrows= num_rows)
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    print("....")
    
    #Transform the Gender into a binary feature with Female = 1, Male = 0 and 'XNA' and 'nan's set equal to 0.5
    #Transform the Yes/No Categories that have not yet been transformed into binary features (yes=1; no=0)
    #Tranform the Contract type into a binary feature (cash loan=1; revolving loan=0)
    #Transform the weekdays into numbers with Monday=0 and Sunday=6
    #Transform the education type into levels 
    specific_categories_dict = {'CODE_GENDER'                : {'cat_type' : 'gender', 'impute_value' : 0.5},
                                'FLAG_OWN_CAR'               : {'cat_type' : 'yes_no'},
                                'FLAG_OWN_REALTY'            : {'cat_type' : 'yes_no'},
                                'EMERGENCYSTATE_MODE'        : {'cat_type' : 'yes_no'},
                                'NAME_CONTRACT_TYPE'         : {'cat_type' : 'loan'},
                                'WEEKDAY_APPR_PROCESS_START' : {'cat_type' : 'weekday'},
                                'NAME_EDUCATION_TYPE'        : {'cat_type' : 'education'},
                               }
    
    for cat, kwargs in specific_categories_dict.items():
        df[cat] = cat_encoder(df[cat], **kwargs)[0]
        
    #Reduce the categories for NAME_TYPE_SUITE into Unaccompagnied, Family and Other
    df['NAME_TYPE_SUITE']=specific_cat_reduction(df['NAME_TYPE_SUITE'], cat_type='suite_type')
    
    #Transform the remaining categorical features into One Hot Features
    df, new_cols = one_hot_encoder(df, nan_as_category)
    
    #Correcting obviously false values to np.nan for the variables containing 'DAYS_'. 
    #They are in general referenced to the application data and therefore need to be smaller than zero.
    #Additionally, they need to be smaller than the days since birth.
    days_cols = [col for col in df.columns if col.startswith('DAYS_')]
    select = (df[days_cols] > 0) | (np.array([df[col] < df['DAYS_BIRTH'] for col in days_cols]).T)
    df[days_cols] = np.where(select, np.nan, df[days_cols])
    
    #Feature Engineering based on "domain knowledge" - features used in other kernels
    # Some simple new features (percentages)
    df['NEW_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['NEW_ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['NEW_PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    #Drop columns that have zero variation
    df.drop(df.columns[df.std(axis=0) == 0], axis=1, inplace=True)
    
    #Drop columns that where found to have little to none importance:
    #drop_cols = ['FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_CONT_MOBILE']
    #df.drop(columns=drop_cols, inplace=True)
    df.columns = pd.Index(['APPL_' + col if col not in ['SK_ID_CURR', 'TARGET'] else col for col in df.columns ])

    
    
    print('After the first preprocesing:')
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    return df

### 3.2.2 Le fichier: bureau et bureau_balance

In [8]:
def preprocess_bureau(filepath, num_rows = None, nan_as_category = True):
    df = pd.read_csv(filepath, nrows= num_rows)
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    print("....")
    
    ind_link = df[['SK_ID_CURR', 'SK_ID_BUREAU']]
    
    #The currency used is to over 99% currency 1, so will will transform this to a binary variable.
    specific_categories_dict = {'CREDIT_CURRENCY' : {'cat_type' : 'currency'}}
    for cat, kwargs in specific_categories_dict.items():
        df[cat] = cat_encoder(df[cat], **kwargs)[0]
    
    #Reduce the number of categories for the credit type, but before that count the number of different credit types
    type_agg = df.groupby(by='SK_ID_CURR').agg({'CREDIT_TYPE' : 'nunique'}).rename(
                columns={'CREDIT_TYPE' : 'CREDIT_TYPE_NUNIQUE'})
    df['CREDIT_TYPE']= specific_cat_reduction(df['CREDIT_TYPE'], 'bureau_credit_type')
    
    #Dropping the data for which the credit update is after the application data for the current loan.
    select = df['DAYS_CREDIT_UPDATE'] <= 0
    df = df[select]
    
    #Setting the time columns that are more than 100 years (=-36524.3 days) before the time of the application to np.nan
    days_cols = [col for col in df.columns if col.startswith('DAYS_')]
    select = (df[days_cols] <= -36524.3) 
    df[days_cols] = np.where(select, np.nan, df[days_cols])
    
    #Correcting the entries that are active, have an actual credit enddate, but not a planned credit enddate to 'Closed'
    select = (df['CREDIT_ACTIVE']=='Active') & (df['DAYS_ENDDATE_FACT'].notnull()) & (df['DAYS_CREDIT_ENDDATE'].isnull())
    df['CREDIT_ACTIVE'] = np.where(select, 'Closed', df['CREDIT_ACTIVE'])
    
    #Feature Engineering: Duration of the Credit
    df['NEW_DURATION_CREDIT_DAYS'] = df['DAYS_ENDDATE_FACT'].fillna(df['DAYS_CREDIT_ENDDATE']) - df['DAYS_CREDIT']
    select = df['NEW_DURATION_CREDIT_DAYS'] < 0
    df['NEW_DURATION_CREDIT_DAYS'] = np.where(select, np.nan, df['NEW_DURATION_CREDIT_DAYS'])
    
    #Transform the categorical features into One Hot Features
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # Aggregations for the numerical features
    num_aggregations = {
        'CREDIT_CURRENCY' : ['mean'],
        'NEW_DURATION_CREDIT_DAYS' : ['min', 'max', 'mean'],
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var', 'size'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
    }
    
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in cat_cols: 
        cat_aggregations[cat] = ['mean', 'sum']
    
    agg_df = df.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    agg_df.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in agg_df.columns.tolist()])
    agg_df = agg_df.join(type_agg, how='left', on='SK_ID_CURR')
    
    del type_agg
    gc.collect()

    # Bureau: Active credits - using only numerical aggregations
    active = df[df['CREDIT_ACTIVE_Active'] == 1]
    red_num_aggs = dict(num_aggregations)
    red_num_aggs.pop('DAYS_ENDDATE_FACT')
    active_agg = active.groupby('SK_ID_CURR').agg(red_num_aggs)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    agg_df = agg_df.join(active_agg, how='left', on='SK_ID_CURR')
    
    del active, active_agg
    gc.collect()
    
    ## Bureau: Closed credits - using only numerical aggregations
    closed = df[df['CREDIT_ACTIVE_Closed'] == 1]
    red_num_aggs = dict(num_aggregations)
    red_num_aggs.pop('DAYS_CREDIT_ENDDATE')
    closed_agg = closed.groupby('SK_ID_CURR').agg(red_num_aggs)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    agg_df = agg_df.join(closed_agg, how='left', on='SK_ID_CURR')
    
    del closed, closed_agg, df
    gc.collect()
    #return bureau_agg
    
    #Drop columns that have zero variation
    agg_df.drop(agg_df.columns[agg_df.std(axis=0) == 0], axis=1, inplace=True)
    
    print('After the first preprocesing and aggregation:')
    print(f"The data contains {len(agg_df)} samples,")
    print(f"and there are {agg_df.shape[1]} features in the dataset.")
    
    return agg_df, ind_link

In [9]:
def preprocess_bureau_balance(filepath, num_rows = None, nan_as_category = True):
    df = pd.read_csv(filepath, nrows= num_rows)
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    print("....")
    
    df['STATUS']= specific_cat_reduction(df['STATUS'], 'bureau_balance_status')
    #impute the values for the unknown status (=np.nan) with the value of the balance the month before
    df = df.sort_values(by=['SK_ID_BUREAU', 'MONTHS_BALANCE'])
    df['STATUS'] = df.groupby('SK_ID_BUREAU')['STATUS'].ffill()
    
    #Determine if the Account was Closed or Not and the month for which the account was closed 
    #status_agg = df.groupby('SK_ID_BUREAU')[['SK_ID_BUREAU']].first().reset_index(drop=True)
    #status_agg = status_agg.join(df[df['STATUS']==-1].groupby('SK_ID_BUREAU').first(), how='left', on='SK_ID_BUREAU')
    #status_agg.columns = pd.Index([e + '_CLOSED' for e in status_agg.columns])
    #status_agg['STATUS_CLOSED'] = status_agg['STATUS_CLOSED'].fillna(0).replace(-1, 1)
    
    
    num_aggregations = {
                        'MONTHS_BALANCE' : ['min', 'max'],
                        'STATUS' : ['max', 'min'],
                        }
    
    agg_df = df.groupby('SK_ID_BUREAU').agg(num_aggregations)
    agg_df.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in agg_df.columns.tolist()])
    #if the minium status is -1, this means that the credit is closed.
    agg_df['BURO_STATUS_MIN'] = agg_df['BURO_STATUS_MIN'].apply(lambda x : 1 if x==-1 else 0)
    agg_df = agg_df.rename(columns={'BURO_STATUS_MIN':'BURO_STATUS_CLOSED'})
    #if the maximum status is -1, replace the value with np.nan. One does not know what the max. balance status was.
    agg_df['BURO_STATUS_MAX'] = agg_df['BURO_STATUS_MAX'].replace(-1, np.nan)
    # determine the month the account was closed
    closed = df['STATUS'] == -1
    agg_df = agg_df.join(df[closed].groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min().rename(
                         'BURO_MONTHS_BALANCE_CLOSED'),
                         how='left', on='SK_ID_BUREAU')
    
    num_aggregations = {
                        'MONTHS_BALANCE' : ['size'],
                        'STATUS' : ['mean'],
                        }
    
    #determine the actual time for which one has information on the balance
    df = df.drop(df[closed].index).dropna(subset=['STATUS'])
    status_agg = df.groupby('SK_ID_BUREAU').agg(num_aggregations)
    status_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in status_agg.columns.tolist()])
    agg_df = agg_df.join(status_agg, how='left', on='SK_ID_BUREAU')
    
    del closed, status_agg, df
    gc.collect()
        
    print('After the first preprocesing and aggregation:')
    print(f"The data contains {len(agg_df)} samples,")
    print(f"and there are {agg_df.shape[1]} features in the dataset.")
    
    return agg_df

In [10]:
def aggregate_bureau_balance(bureau, balance, bureau_balance_link):
    num_aggregations = {
                        'BURO_MONTHS_BALANCE_MIN' : ['min'],
                        'BURO_MONTHS_BALANCE_MAX' : ['max'],
                        'BURO_STATUS_MAX' : ['max'],
                        'BURO_STATUS_CLOSED' : ['mean', 'sum'],
                        'BURO_MONTHS_BALANCE_CLOSED' : ['mean'],
                        'BURO_MONTHS_BALANCE_SIZE' : ['mean'],
                        'BURO_STATUS_MEAN' : ['min','max','mean'],
                        }

    df = bureau_balance_link.join(balance, how='left', on='SK_ID_BUREAU')
    balance_agg = df.groupby('SK_ID_CURR').agg(num_aggregations)
    balance_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in balance_agg.columns.tolist()])
    
    balance_agg = bureau.join(balance_agg, how='left', on='SK_ID_CURR')
    
    del df
    gc.collect()
    
    print('After aggregating bureau and bureau_balance:')
    print(f"The data contains {len(balance_agg)} samples,")
    print(f"and there are {balance_agg.shape[1]} features in the dataset.")
    
    return balance_agg

### 3.2.3 Le fichier: previous_applications

In [11]:
# Preprocess previous_applications.csv
def preprocess_previous_applications(filepath, num_rows = None, nan_as_category = True, feature_drop=True):
    df = pd.read_csv(filepath, nrows= num_rows)
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    print("....")
    
    #Correcting obviously false values to np.nan for the variables containing 'DAYS_'. 
    #We limit them to being less than 100 years in the future and require them to be after decision date for the loan.
    days_cols = [col for col in df.columns if col.startswith('DAYS_')]
    select = (df[days_cols] > 35624.3) | (np.array([df[col] < df['DAYS_DECISION'] for col in days_cols]).T)
    df[days_cols] = np.where(select, np.nan, df[days_cols])
    
    #Treating categories that can transform to numerical values
    specific_categories_dict = {'FLAG_LAST_APPL_PER_CONTRACT': {'cat_type' : 'yes_no'},
                                'WEEKDAY_APPR_PROCESS_START' : {'cat_type' : 'weekday'},
                                'NAME_YIELD_GROUP'           : {'cat_type' : 'yield_group'},
                               }
    
    for cat, kwargs in specific_categories_dict.items():
        df[cat] = cat_encoder(df[cat], **kwargs)[0]
        
    #Reduce the categories for NAME_TYPE_SUITE into Unaccompagnied, Family, Childre, Partner/Spouse and Other
    df['NAME_TYPE_SUITE']=specific_cat_reduction(df['NAME_TYPE_SUITE'], cat_type='suite_type')
        
    #A small category reduction for 'NAME_PAYMENT_TYPE':
    df['NAME_PAYMENT_TYPE'] = specific_cat_reduction(df['NAME_PAYMENT_TYPE'], cat_type='payment_type')
    
    #The information of category 'NAME_CONTRACT_STATUS' is basically contained in 'CODE_REJECT_REASON'.
    #We can combine these two into a single category.
    select1 = df['NAME_CONTRACT_STATUS'] == 'Approved' #APPROVED
    select2 = df['NAME_CONTRACT_STATUS'] == 'Canceled' #CANCELED
    select3 = df['CODE_REJECT_REASON'] == 'XAP' #XNA
    df['CODE_CONTRACT_STATUS'] = np.where(select1, 'APPROVED', 
                                 np.where(select2, 'CANCELED', 
                                 np.where(select3, 'XNA', df['CODE_REJECT_REASON'])))
    df.drop(columns=['NAME_CONTRACT_STATUS', 'CODE_REJECT_REASON'], inplace=True)

    #The categories: 'NAME_CASH_LOAN_PURPOSE' and 'NAME_GOODS_CATEGORY' are mutually explusive
    #The goods categories is for Consumer loans. We combine these two categories to create a general purpose category.
    #Since there are many categories we will do some category reduction.
    df['LOAN_PURPOSE'] = specific_cat_reduction(df['NAME_CASH_LOAN_PURPOSE'], cat_type = 'loan_purpose').fillna(
                         specific_cat_reduction(df['NAME_GOODS_CATEGORY'], cat_type = 'loan_purpose'))
    df.drop(columns=['NAME_CASH_LOAN_PURPOSE', 'NAME_GOODS_CATEGORY'], inplace=True)
    
    #The category 'NAME_PORTFOLIO' does not seem to bring much more information. 
    #It is XNA, when the yield group 'NAME_YIELD_GROUP' is XNA. Otherwise it corresponds to 'NAME_CONTRACT_TYPE', 
    #with the exception of 'Cars', which is less than 500 samples. (we can drop this category)
    df.drop(columns=['NAME_PORTFOLIO'], inplace=True)
    
    #The category 'PRODUCT_COMBINATION' is essentially 'NAME_CONTRACT_TYPE' + 'NAME_PRODUCT_TYPE' + 'NAME_YIELD_GROUP'
    #These is additional information for the Consumer loans ('househhold', 'mobile', 'industry', 'other') 
    #and 'with / without interest'. 
    #At the same time the 'NAME_PRODUCT_TYPE' only refers to Cash and Revolving loans.
    select = df['NAME_CONTRACT_TYPE'] == 'Consumer loans'
    df['NAME_PRODUCT_TYPE'] = np.where(select, df['PRODUCT_COMBINATION'], df['NAME_PRODUCT_TYPE'])
    df.drop(columns=['PRODUCT_COMBINATION'], inplace=True)

    #We will flag the entries without a yield group.
    df['FLAG_YIELD_GROUP'] = df['NAME_YIELD_GROUP'].isnull().astype(int)

    #categories that were found to have very little importance
    if feature_drop:
        df.drop(columns=['CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY', 'LOAN_PURPOSE',
                         'NAME_PRODUCT_TYPE'],
                inplace=True)
    
    #Transform the remaining categorical features into One Hot Features
    df, cat_cols = one_hot_encoder(df, nan_as_category)    
    
    # Add feature: value ask / value received percentage
    df['NEW_APPL_CREDIT_PERC'] = df['AMT_APPLICATION'] / df['AMT_CREDIT']
    

        
    # Previous applications numeric features, including the categorical features transformed into numbers
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'NEW_APPL_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'WEEKDAY_APPR_PROCESS_START' : ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'RATE_INTEREST_PRIMARY' : ['mean'],
        'RATE_INTEREST_PRIVILEGED' : ['mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'SELLERPLACE_AREA' : ['mean', pd.Series.mode],
        'NAME_YIELD_GROUP' : ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'DAYS_FIRST_DRAWING' : ['min', 'max', 'mean'],
        'DAYS_FIRST_DUE' : ['min', 'max', 'mean'],
        'DAYS_LAST_DUE_1ST_VERSION' : ['min', 'max', 'mean'],
        'DAYS_LAST_DUE' : ['min', 'max', 'mean'],
        'DAYS_TERMINATION' : ['min', 'max', 'mean'],
        'NFLAG_INSURED_ON_APPROVAL' : ['mean', 'sum'],
        'FLAG_YIELD_GROUP' : ['mean']
    }
    

    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean', 'sum']
        
    agg_df = df.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    agg_df.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in agg_df.columns.tolist()])
    
    num_aggregations = {
            'AMT_APPLICATION': ['min', 'max', 'mean', 'sum'],
            'NEW_APPL_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
            'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'sum'],
            'HOUR_APPR_PROCESS_START': ['min', 'max'],
            'WEEKDAY_APPR_PROCESS_START' : ['min', 'max', 'mean'],
            'DAYS_DECISION': ['min', 'max', 'sum'],
            'NAME_YIELD_GROUP' : ['min', 'max', 'mean'],
        }   
    
    
    # Previous Applications: Approved Applications - only numerical features
    approved = df[df['CODE_CONTRACT_STATUS_APPROVED'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    agg_df = agg_df.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    # After some tests they don's seem to contribute much therefore we will take this part out.
    if not feature_drop :
        select_refused = (df['CODE_CONTRACT_STATUS_APPROVED'] == 0) | (df['CODE_CONTRACT_STATUS_CANCELED'] == 0) | \
                         (df['CODE_CONTRACT_STATUS_CLIENT'] == 0) | (df['CODE_CONTRACT_STATUS_XNA'] == 0) 
        refused = df[select_refused]
        refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
        refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
        agg_df = agg_df.join(refused_agg, how='left', on='SK_ID_CURR')
        del refused, refused_agg
    
    del approved, approved_agg, df
    gc.collect()
    
    #Drop columns that have zero variation
    for col in agg_df.select_dtypes(include = 'object'):
        try:
            agg_df[col] = pd.to_numeric(agg_df[col])
        except:
            print(f"ATTENTION : Column {col} is coerced to be numeric!")
            agg_df[col] = pd.to_numeric(agg_df[col], errors='coerce')
            
    agg_df.drop(agg_df.columns[agg_df.std(axis=0) == 0], axis=1, inplace=True)
    
    print('After the first preprocesing and aggregation:')
    print(f"The data contains {len(agg_df)} samples,")
    print(f"and there are {agg_df.shape[1]} features in the dataset.")
    
    
    return agg_df

### 3.2.4 Le fichier: POS_CASH_balance

In [12]:
def preprocess_pos_cash(filepath, num_rows = None, nan_as_category = True):
    df = pd.read_csv(filepath, nrows= num_rows)
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    print("....")
    df, cat_cols = one_hot_encoder(df, nan_as_category)


    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'sum', 'var', 'size'],
        'SK_DPD':         ['max', 'mean'],
        'SK_DPD_DEF':     ['max', 'mean'],
        'CNT_INSTALMENT': ['mean', 'sum'],
        'CNT_INSTALMENT_FUTURE' : ['mean','sum'],
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean', 'sum']
    
    agg_df = df.groupby('SK_ID_CURR').agg(aggregations)
    agg_df.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in agg_df.columns.tolist()])
    # Count pos cash accounts
    #agg_df['POS_COUNT'] = df.groupby('SK_ID_CURR').size()
    
    del df
    gc.collect()
    
    #Drop columns that have zero variation
    agg_df.drop(agg_df.columns[agg_df.std(axis=0) == 0], axis=1, inplace=True)

    
    print('After the first preprocesing and aggregation:')
    print(f"The data contains {len(agg_df)} samples,")
    print(f"and there are {agg_df.shape[1]} features in the dataset.")
    
    return agg_df

### 3.2.5 Le fichier: installments_payments

In [13]:
def preprocess_installments_payments(filepath, num_rows = None, nan_as_category = True):
    df = pd.read_csv(filepath, nrows= num_rows)
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    print("....")

    # Percentage and difference paid in each installment (amount paid and installment value)
    df['NEW_PAYMENT_PERC'] = df['AMT_PAYMENT'] / df['AMT_INSTALMENT']
    df['NEW_PAYMENT_DIFF'] = df['AMT_INSTALMENT'] - df['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    df['NEW_DPD'] = df['DAYS_ENTRY_PAYMENT'] - df['DAYS_INSTALMENT']
    df['NEW_DBD'] = df['DAYS_INSTALMENT'] - df['DAYS_ENTRY_PAYMENT']
    df['NEW_DPD'] = df['NEW_DPD'].apply(lambda x: x if x > 0 else 0)
    df['NEW_DBD'] = df['NEW_DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Features
    aggregations = {
        'NEW_DPD': ['max', 'mean', 'sum'],
        'NEW_DBD': ['max', 'mean', 'sum'],
        'NEW_PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'NEW_PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    
    agg_df = df.groupby('SK_ID_CURR').agg(aggregations)
    agg_df.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in agg_df.columns.tolist()])
    # Count pos cash accounts
    agg_df['INSTAL_COUNT'] = df.groupby('SK_ID_CURR').size()
    
    del df
    gc.collect()
    
    #Drop columns that have zero variation
    agg_df.drop(agg_df.columns[agg_df.std(axis=0) == 0], axis=1, inplace=True)
    
    print('After the first preprocesing and aggregation:')
    print(f"The data contains {len(agg_df)} samples,")
    print(f"and there are {agg_df.shape[1]} features in the dataset.")
    
    return agg_df

### 3.2.6 Le fichier: credit_card_balance

In [14]:
def preprocess_credit_card_balance(filepath, num_rows = None, nan_as_category = True):
    df = pd.read_csv(filepath, nrows= num_rows)
    print(f"The data contains {len(df)} samples.")
    print(f"and there are {df.shape[1]} features in the dataset.")
    print("....")
    df, cat_cols = one_hot_encoder(df, nan_as_category)

    # Percentage and difference paid each (amount paid and balance amount)
    df['NEW_PAYMENT_PERC'] = df['AMT_PAYMENT_CURRENT'] / df['AMT_BALANCE']
    df['NEW_PAYMENT_DIFF'] = df['AMT_BALANCE'] - df['AMT_PAYMENT_CURRENT']
    

    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'AMT_BALANCE' :   ['min', 'max', 'mean', 'sum', 'var'],
        'AMT_CREDIT_LIMIT_ACTUAL' : ['min', 'max', 'mean', 'sum', 'var'],
        'AMT_DRAWINGS_ATM_CURRENT' : ['max','mean','sum','var'],
        'AMT_DRAWINGS_CURRENT' : ['max','mean','sum','var'],
        'AMT_DRAWINGS_OTHER_CURRENT' : ['max','mean','sum','var'],
        'AMT_DRAWINGS_POS_CURRENT' : ['max','mean','sum','var'],
        'AMT_INST_MIN_REGULARITY' : ['min','max','mean','sum','var'],
        'AMT_PAYMENT_CURRENT' :  ['min','max','mean','sum','var'],
        'AMT_PAYMENT_TOTAL_CURRENT' : ['min','max','mean','sum','var'],
        'AMT_RECEIVABLE_PRINCIPAL' : ['min','max','mean','sum','var'],
        'AMT_RECIVABLE' : ['min','max','mean','sum','var'],
        'AMT_TOTAL_RECEIVABLE' : ['min','max','mean','sum','var'],
        'CNT_DRAWINGS_ATM_CURRENT' : ['mean', 'sum'],
        'CNT_DRAWINGS_CURRENT' : ['mean', 'sum'],
        'CNT_DRAWINGS_POS_CURRENT' : ['mean', 'sum'],
        'CNT_INSTALMENT_MATURE_CUM' : ['mean','sum'],
        'NEW_PAYMENT_PERC' : ['max', 'mean', 'sum', 'var'],
        'NEW_PAYMENT_DIFF' : ['max', 'mean', 'sum', 'var'],
        'SK_DPD':         ['max', 'mean'],
        'SK_DPD_DEF':     ['max', 'mean'],

    }
    for cat in cat_cols:
        aggregations[cat] = ['mean', 'sum']
    
    agg_df = df.groupby('SK_ID_CURR').agg(aggregations)
    agg_df.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in agg_df.columns.tolist()])
    #Count credit card balance
    agg_df['CC_COUNT'] = df.groupby('SK_ID_CURR').size()
    
    
    del df
    gc.collect()
    
    #Drop columns that have zero variation
    agg_df.drop(agg_df.columns[agg_df.std(axis=0) == 0], axis=1, inplace=True)

    
    print('After the first preprocesing and aggregation:')
    print(f"The data contains {len(agg_df)} samples,")
    print(f"and there are {agg_df.shape[1]} features in the dataset.")
    
    return agg_df

### 3.2.7 Fonction pour l'Aggrégation

In [15]:
def join_with_application(application, other):
    df = application.join(other, how='left', on='SK_ID_CURR')
    
    print('After joining the two datasets:')
    print(f"The data contains {len(df)} samples,")
    print(f"and there are {df.shape[1]} features in the dataset.")
    
    return df

## 3.3 Import des Données et Execution du Prétraitement

In [16]:
def get_shortcut_path(filepath):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(filepath)
    return shortcut.Targetpath +'\\'

In [17]:
def load_and_preprocess_data(path):
    filenames = ['application_train', 'application_test', 'previous_application',
                 'credit_card_balance', 'POS_CASH_balance', 'installments_payments',
                 'bureau', 'bureau_balance']
    preprocessing_dict = {'application_train' : preprocess_application,
                          'application_test' : preprocess_application,
                          'previous_application' : lambda x: preprocess_previous_applications(x, feature_drop=False),
                          'credit_card_balance' : preprocess_credit_card_balance,
                          'POS_CASH_balance' : preprocess_pos_cash,
                          'installments_payments' : preprocess_installments_payments,
                          'bureau' : preprocess_bureau,
                          'bureau_balance' : preprocess_bureau_balance}
    data = {}
    for filename in filenames:
        filepath = path + filename +'.csv'
        print(f"Loading and Preprocessing {filename.upper()} ...")
        if filename != 'bureau':
            data[filename] = preprocessing_dict[filename](filepath)
        else:
            data[filename], bureau_link = preprocessing_dict[filename](filepath)
        print()
    data['bureau'] = aggregate_bureau_balance(data['bureau'], data['bureau_balance'], bureau_link)
    print(50*'*')
    print("Joining the DATA")
    print()
    application = data['application_train']
    application_submit = data['application_test']
    for other in ['previous_application', 'credit_card_balance', 'POS_CASH_balance', 'installments_payments', 'bureau'] :
        print(f"Joining APPLICATION with {filename.upper()} data :")
        application = join_with_application(application, data[other])
        print()
        print(f"Joining APPLICATION_SUBMIT with {filename.upper()} data :")
        application_submit = join_with_application(application_submit, data[other])
        print()
    print(50*'*')
    print("DONE")
    del data
    gc.collect()
    return application, application_submit

# 4. Import des Données et Exécution du Prétraitement

In [18]:
PATH = os.getcwd()
DATAFOLDER = '\\DataShortcut.lnk'
filepath = get_shortcut_path(PATH+DATAFOLDER)

In [19]:
application, application_submit = load_and_preprocess_data(filepath)

Loading and Preprocessing APPLICATION_TRAIN ...
The data contains 307511 samples.
and there are 122 features in the dataset.
....
After the first preprocesing:
The data contains 307511 samples.
and there are 237 features in the dataset.

Loading and Preprocessing APPLICATION_TEST ...
The data contains 48744 samples.
and there are 121 features in the dataset.
....
After the first preprocesing:
The data contains 48744 samples.
and there are 223 features in the dataset.

Loading and Preprocessing PREVIOUS_APPLICATION ...
The data contains 1670214 samples.
and there are 37 features in the dataset.
....
ATTENTION : Column PREV_SELLERPLACE_AREA_MODE is coerced to be numeric!
After the first preprocesing and aggregation:
The data contains 338857 samples,
and there are 266 features in the dataset.

Loading and Preprocessing CREDIT_CARD_BALANCE ...
The data contains 3840312 samples.
and there are 23 features in the dataset.
....
After the first preprocesing and aggregation:
The data contains 10

In [20]:
print(application.shape)
print(application_submit.shape)

(307511, 775)
(48744, 761)


In [21]:
y = application['TARGET']

print(f"{len(application[y==1])} clients in the dataset defaulted, {len(application[y==0])} did not.")
print(f"That's {round(100*len(application[y==1])/len(application[y==0]), 2)}%.")

24825 clients in the dataset defaulted, 282686 did not.
That's 8.78%.


# 5. Export des Données Prétraitées

In [23]:
export = True
sample = True
if export:
    PATH = os.getcwd()
    DATAFOLDER = '\\Data\\'

    sufffix=''
    if sample:
        for filename, data in zip(['data_processed_min', 'data_processed_submit_min'], 
                                  [application.sample(frac=0.09), application_submit.sample(frac=0.5)]) :
            data.to_csv(PATH + DATAFOLDER + filename + '.csv')
    else:
        for filename, data in zip(['data_processed', 'data_processed_submit'], 
                                  [application, application_submit]) :
            data.to_csv(PATH + DATAFOLDER + filename + '.csv')